# Rough

In [2]:
# Initialization block
from IPython.display import display, HTML, Image , Markdown
from snowflake.snowpark.session import Session
import snowflake.snowpark.types as T
import snowflake.snowpark.functions as F
import os ,configparser ,json ,logging

# Import the commonly defined utility scripts using
# dynamic path include
import sys
sys.path.append('../python/lutils')
import sflk_base as L

display(Markdown("### Initialization"))
logging.basicConfig(stream=sys.stdout, level=logging.ERROR)

# Source various helper functions
%run ./scripts/notebook_helpers.py

# Define the project home directory, this is used for locating the config.ini file
PROJECT_HOME_DIR = '../../'
config = L.get_config(PROJECT_HOME_DIR)
sp_session = L.connect_to_snowflake(PROJECT_HOME_DIR)

if(sp_session == None):
    raise Exception(f'Unable to connect to snowflake. Validate connection information ')

sp_session.use_role(f'''{config['APP_DB']['role']}''')
sp_session.use_schema(f'''{config['APP_DB']['database']}.{config['APP_DB']['schema']}''')
sp_session.use_warehouse(f'''{config['SNOW_CONN']['warehouse']}''')

df = sp_session.sql('select current_user() ,current_role() ,current_database() ,current_schema();').to_pandas()
display(df)

### Initialization

,CURRENT_USER(),CURRENT_ROLE(),CURRENT_DATABASE(),CURRENT_SCHEMA()
0,VSEKAR,PUBLIC,INDSOL_CMSGOV_PRICING,PUBLIC


In [3]:
import pandas as pd
import os ,re

pd.set_option('display.max_colwidth', None)

def get_basename_of_datafile(p_datafile:str) -> str:
    base = os.path.basename(p_datafile)
    fl_base = os.path.splitext(base)
    return fl_base[0]

def get_cleansed_file_basename(p_datafile):
    fl_basename = get_basename_of_datafile(p_datafile)
    # Replace all non alphanumeric characters with _
    fl_name = re.sub('[^0-9a-zA-Z]+', '_', fl_basename)
    return fl_name

# INPUT_DATA_STAGE = config['APP_DB']['ext_stage']
INPUT_DATA_STAGE = 'data_stg'
DATA_STAGE_FOLDER = config['APP_DB']['folder_data']

DATA_FILE = '2022_10_01_priority_health_HMO_in-network-rates.zip'
DATA_FILE = 'reduced_sample_data.json'

DATA_FILE_BASENAME = get_basename_of_datafile(DATA_FILE)
DATA_FILE_BASENAME_CLEANSED = get_cleansed_file_basename(DATA_FILE)

TARGET_DATA_STAGE = config['APP_DB']['ext_stage']
TARGET_FOLDER = config['APP_DB']['folder_parsed']

In [9]:
df = sp_session.table('in_network_rates_file_header').select('data_file' ,'INSERTED_AT').sort(F.col('INSERTED_AT'), ascending=False).distinct()
for r in df.to_local_iterator():
    print(r['DATA_FILE'])

reduced_sample_data.json
2022_10_01_priority_health_HMO_in-network-rates.zip


In [ ]:
display(Markdown("The table segment_task_execution_status contains the audit of execution for the above stored procedures"))

df = (sp_session.table('segment_task_execution_status')
        .filter(F.col('DATA_FILE') == F.lit(DATA_FILE))
        .sample(n=5)
        .to_pandas())

display(df)

--- 
### Closeout

    With that we are finished this section of the demo setup

In [ ]:
sp_session.close()
print('Finished!!!')